In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import logging
logging.basicConfig(filename="./sales_table_log.log", level=logging.INFO)

def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_sturday_ty = (datetime.datetime.now()-datetime.timedelta(days=(datetime.datetime.now().weekday()+2))).date()
first_week_ty=datetime.date(2020,3,1)

last_sturday_ly = last_sturday_ty-datetime.timedelta(days=52*7)
first_week_ly=first_week_ty-datetime.timedelta(days=52*7)



In [2]:
last_sturday_ty,first_week_ty,last_sturday_ly,first_week_ly

(datetime.date(2020, 4, 25),
 datetime.date(2020, 3, 1),
 datetime.date(2019, 4, 27),
 datetime.date(2019, 3, 3))

In [3]:
list_POS_this_year=list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/"))
list_POS_this_year=[x for x in list_POS_this_year if "daily" in x.lower()]

list_POS_this_year=[x for x in list_POS_this_year if x.split("/MediaStorm_")[1][:10]>=str(first_week_ty)]
list_POS_this_year=[x for x in list_POS_this_year if x.split("/MediaStorm_")[1][:10]<=str(last_sturday_ty)]

# 
list_POS_last_year=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
list_POS_last_year=[x for x in list_POS_last_year if "daily" in x.lower()]

list_POS_last_year=[x for x in list_POS_last_year if x.split("/MediaStorm_")[1][:10]>=str(first_week_ly)]
list_POS_last_year=[x for x in list_POS_last_year if x.split("/MediaStorm_")[1][:10]<=str(last_sturday_ly)]

In [4]:
df_2020=pd.DataFrame()
df_prod_mapping=pd.read_csv("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20200401-135137-346.txt",
                            sep="|",dtype=str,usecols=['division_id','department_id','class_code_id','subclass_id'])
for file in list_POS_this_year:
    df=pd.read_csv(file,dtype=str,sep="|")
    df=pd.merge(df,df_prod_mapping,on=['class_code_id','subclass_id'],how="left")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['division_id']=df['division_id'].fillna("-1")
    df['department_id']=df['department_id'].fillna("-1")
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"non_rewards","rewards")
    df['store_label']=np.where(df['location_id']=="6990","online","instore")
    week_end_dt=df['transaction_dt'].max()
    
    df_sales=df.groupby(['division_id','department_id','rewards_label','store_label'])['item_transaction_amt'].sum().to_frame().reset_index()
    
    df_trans=df[['location_id','transaction_dt','transaction_id','rewards_label','division_id','department_id','store_label']].drop_duplicates()
    df_trans=df_trans.groupby(['division_id','department_id','rewards_label','store_label'])['transaction_id'].count().to_frame().reset_index()
    df=pd.merge(df_sales,df_trans,on=['division_id','department_id','rewards_label','store_label'],how="outer")
    
    df['week_end_dt']=week_end_dt
    df=df.rename(columns={"item_transaction_amt":"sales","transaction_id":"trans"})
    df_2020=df_2020.append(df)
    print(week_end_dt,datetime.datetime.now())
    

2020-03-07 2020-04-27 12:22:53.410848
2020-03-28 2020-04-27 12:23:50.485966
2020-03-14 2020-04-27 12:25:43.471407
2020-03-21 2020-04-27 12:27:36.511691
2020-04-04 2020-04-27 12:28:52.141509
2020-04-11 2020-04-27 12:30:18.657948
2020-04-18 2020-04-27 12:31:30.108791


In [5]:
df_2019=pd.DataFrame()
df_prod_mapping=pd.read_csv("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190401-134939-109.txt",
                            sep="|",dtype=str,usecols=['division_id','department_id','class_code_id','subclass_id'])
for file in list_POS_last_year:
    df=pd.read_csv(file,dtype=str,sep="|")
    df=pd.merge(df,df_prod_mapping,on=['class_code_id','subclass_id'],how="left")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['division_id']=df['division_id'].fillna("-1")
    df['department_id']=df['department_id'].fillna("-1")
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"non_rewards","rewards")
    df['store_label']=np.where(df['location_id']=="6990","online","instore")
    week_end_dt=df['transaction_dt'].max()
    
    df_sales=df.groupby(['division_id','department_id','rewards_label','store_label'])['item_transaction_amt'].sum().to_frame().reset_index()
    
    df_trans=df[['location_id','transaction_dt','transaction_id','rewards_label','division_id','department_id','store_label']].drop_duplicates()
    df_trans=df_trans.groupby(['division_id','department_id','rewards_label','store_label'])['transaction_id'].count().to_frame().reset_index()
    df=pd.merge(df_sales,df_trans,on=['division_id','department_id','rewards_label','store_label'],how="outer")
    
    df['week_end_dt']=week_end_dt
    df=df.rename(columns={"item_transaction_amt":"sales","transaction_id":"trans"})
    df_2019=df_2019.append(df)
    print(week_end_dt,datetime.datetime.now())
    

2019-03-09 2020-04-27 12:32:37.880470
2019-03-16 2020-04-27 12:33:43.744556
2019-03-23 2020-04-27 12:34:48.273265
2019-03-30 2020-04-27 12:35:51.650136
2019-04-06 2020-04-27 12:37:13.175329
2019-04-13 2020-04-27 12:38:36.593070
2019-04-20 2020-04-27 12:39:49.663506
2019-04-27 2020-04-27 12:40:47.395367


In [6]:
print("df_2019.shape",df_2019.shape)
print("df_2020.shape",df_2020.shape)

df_2019['division_id']=df_2019['division_id'].astype(int)
df_2020['division_id']=df_2020['division_id'].astype(int)
df_2019['department_id']=df_2019['department_id'].astype(int)
df_2020['department_id']=df_2020['department_id'].astype(int)

df_2019=df_2019.sort_values(['division_id','department_id','rewards_label','store_label'])
df_2020=df_2020.sort_values(['division_id','department_id','rewards_label','store_label'])


df_2019.shape (1240, 7)
df_2020.shape (1135, 7)


In [7]:
df_2019_wide=df_2019.pivot_table(index=['division_id','department_id','rewards_label','store_label'],columns="week_end_dt",values="sales").reset_index()
df_2020_wide=df_2020.pivot_table(index=['division_id','department_id','rewards_label','store_label'],columns="week_end_dt",values="sales").reset_index()
df_2019_wide.insert(0,'year',"2019")
df_2020_wide.insert(0,'year',"2020")

In [15]:
for col in df_2019_wide.columns.tolist():
    if "-" in col:
        df_2019_wide[col]=df_2019_wide[col].fillna(0)
        
for col in df_2020_wide.columns.tolist():
    if "-" in col:
        df_2020_wide[col]=df_2020_wide[col].fillna(0)

In [9]:
division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|")
department_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",sep="|")


In [20]:
writer=pd.ExcelWriter("./BL_sales_by_department_week_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_2019_wide.to_excel(writer,"2019_wide_all",index=False)
df_2020_wide.to_excel(writer,"2020_wide_all",index=False)
division_name.to_excel(writer,"division_name",index=False)
department_name.to_excel(writer,"department_name",index=False)
df_2019.to_excel(writer,"2019_long_all",index=False)
df_2020.to_excel(writer,"2020_long_all",index=False)
for comb,group in df_2019_wide.groupby(['rewards_label','store_label']):
    year=group['year'].unique().tolist()[0]
    tab_name="_".join(comb)+"_"+year
    group.to_excel(writer,tab_name,index=False)
    
for comb,group in df_2020_wide.groupby(['rewards_label','store_label']):
    year=group['year'].unique().tolist()[0]
    tab_name="_".join(comb)+"_"+year
    group.to_excel(writer,tab_name,index=False)
writer.save()